In [18]:
# Using cosine similarity
import graph2vec as g2v
import motif_finder as mf
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle
n_dimensions = 128

def get_most_similar(motif_dict, motif):
    cos_sim_list = []
    for m in motif_dict:
        cos_sim = cosine_similarity(list(motif_dict[m].values())[0].reshape(1,-1), motif.reshape(1,-1))
        cos_sim = cos_sim.tolist()[0][0]
        cos_sim_list.append(cos_sim)
    sim_motif = list(motif_dict.keys())[cos_sim_list.index(min(cos_sim_list))]
    sim_motif_embedding = list(motif_dict[sim_motif].values())[0]
    return [sim_motif,sim_motif_embedding]

def motif_merging1(input_file_motif_clusters='motifs_by_cluster.pickle', cluster_id=0, min_sim=0.005):
    with open(input_file_motif_clusters, 'rb') as pickle_in:
        motifs_by_cluster = pickle.load(pickle_in)
        
    motif_dict = motifs_by_cluster[cluster_id]
    
    m2vModel = g2v.Graph2Vec(size=n_dimensions)
    m2vModel = m2vModel.fit_transform(list(motif_dict.keys()))
    # m2vModel.save_embeddings(len(motif_dict), output_path='./results/motif_embeddings.csv')
    motif_embedding = pd.read_csv('embeddings.csv', index_col=0)
    
    i = 0
    for motif in motif_dict:
        motif_dict[motif]={motif_dict[motif]:motif_embedding.values[i]}
        i+=1
        
    new_dict = {}
    merge_dict = {}
    i = 0
    for motif in motif_dict:
        if len(new_dict)==0:
            new_dict[motif] = motif_dict[motif]
            merge_dict.update({i:[]})
        else:
            m_embedding = list(motif_dict[motif].values())[0]
            sim_motif = get_most_similar(new_dict, m_embedding)
            cos_sim = cosine_similarity(sim_motif[1].reshape(1,-1), m_embedding.reshape(1,-1))
            if cos_sim < min_sim:
                new_dict[sim_motif[0]] = {(list(new_dict[sim_motif[0]].keys())[0]+list(motif_dict[motif].keys())[0]):sim_motif[1]}
                sim_motif_index = list(motif_dict.keys()).index(sim_motif[0])
                merge_dict[sim_motif_index].append(i)
            else:
                new_dict[motif] = motif_dict[motif]
                merge_dict.update({i:[]})
        i+=1
        
    return new_dict, merge_dict

In [19]:
motif_merging1()

100%|██████████| 28/28 [00:00<00:00, 1331.28it/s]
/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2019-05-17 04:26:31,258 : INFO : collecting all words and their counts
2019-05-17 04:26:31,261 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-05-17 04:26:31,269 : INFO : collected 472 word types and 28 unique tags from a corpus of 28 examples and 1540 words
2019-05-17 04:26:31,272 : INFO : Loading a fresh vocabulary
2019-05-17 04:26:31,275 : INFO : effective_min_count=5 retains 65 unique words (13% of original 472, drops 407)
2019-05-17 04:26:31,281 : INFO : effective_min_count=5 leaves 1039 word corpus (67% of original 1540, drops 501)
2019-05-17 04:26:31,285 : INFO : deleting the raw counts dictionary of 472 items
2019-05-1

2019-05-17 04:26:31,434 : INFO : EPOCH - 8 : training on 1540 raw words (104 effective words) took 0.0s, 6983 effective words/s
2019-05-17 04:26:31,443 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-05-17 04:26:31,444 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-05-17 04:26:31,446 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-05-17 04:26:31,447 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-05-17 04:26:31,448 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-05-17 04:26:31,449 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-17 04:26:31,449 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-17 04:26:31,455 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-17 04:26:31,457 : INFO : EPOCH - 9 : training on 1540 raw words (124 effective words) took 0.0s, 8665 effective words/s
2019-05-17 04:26

({<networkx.classes.digraph.DiGraph at 0x1a1c641cf8>: {612: array([ 4.70288534e-04, -1.81506609e-03, -1.64279703e-03, -2.37680436e-03,
           7.15065806e-04, -3.19029600e-03,  2.74153845e-03,  2.66209105e-03,
           2.09974078e-03,  1.60401734e-03,  1.87162100e-03,  1.43138721e-04,
           2.71101901e-03, -3.28794261e-03, -3.34815029e-03, -1.64314441e-03,
          -1.72375888e-03,  1.13912357e-03, -3.33082280e-03, -8.13924707e-04,
           3.08022182e-03,  2.13612919e-03,  2.20213388e-03,  3.83029389e-03,
          -1.89529755e-03,  2.96985544e-03, -1.54142710e-03,  3.72909359e-03,
          -2.56763189e-04,  3.66044743e-03, -2.17247265e-03,  3.07010981e-04,
          -2.14087940e-03, -2.55115889e-03,  2.06458778e-03,  3.12204054e-03,
          -1.56668888e-03, -2.76262988e-03,  3.70021304e-03,  1.40838511e-03,
          -1.06384396e-03, -1.49345992e-03,  3.00143799e-03, -9.70628578e-04,
           1.74834640e-04, -1.05069834e-03, -5.72339341e-04,  3.35370447e-03,
       

In [9]:
# Using `get_embedding_clusters`, clustering
# motif_by_clusters: a dictionary where the keys are the cluster labels and the values are lists of motif indices within that cluster.
import graph2vec as g2v
import motif_finder as mf
import pickle
n_dimensions = 128

def motif_merging2(input_file_motif_clusters='motifs_by_cluster.pickle', cluster_id=0):
    with open(input_file_motif_clusters, 'rb') as pickle_in:
        motifs_by_cluster = pickle.load(pickle_in)

    motif_dict = motifs_by_cluster[cluster_id]

    m2vModel = g2v.Graph2Vec(size=n_dimensions)
    m2vModel = m2vModel.fit_transform(list(motif_dict.keys()))
    #m2vModel.save_embeddings(len(motif_dict), output_path='./results/motif_embeddings.csv')
    clusters_of_motif = mf.get_embedding_clusters(embedding_input_file='embeddings.csv', k_for_clustering=10, random_state=None,
                           output_file='clusters_of_motif.pickle')

    freq_by_clusters = {}
    for cluster in clusters_of_motif:
        freq_by_clusters[cluster] = sum([list(motif_dict.values())[i] for i in clusters_of_motif[cluster]])

    return clusters_of_motif, freq_by_clusters

In [10]:
motif_merging2()

100%|██████████| 28/28 [00:00<00:00, 2786.12it/s]
/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2019-05-17 04:24:52,765 : INFO : collecting all words and their counts
2019-05-17 04:24:52,766 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-05-17 04:24:52,767 : INFO : collected 462 word types and 28 unique tags from a corpus of 28 examples and 1540 words
2019-05-17 04:24:52,767 : INFO : Loading a fresh vocabulary
2019-05-17 04:24:52,768 : INFO : effective_min_count=5 retains 65 unique words (14% of original 462, drops 397)
2019-05-17 04:24:52,769 : INFO : effective_min_count=5 leaves 1039 word corpus (67% of original 1540, drops 501)
2019-05-17 04:24:52,770 : INFO : deleting the raw counts dictionary of 462 items
2019-05-1

2019-05-17 04:24:52,934 : INFO : EPOCH - 8 : training on 1540 raw words (104 effective words) took 0.0s, 8777 effective words/s
2019-05-17 04:24:52,945 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-05-17 04:24:52,947 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-05-17 04:24:52,948 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-05-17 04:24:52,949 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-05-17 04:24:52,949 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-05-17 04:24:52,950 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-17 04:24:52,951 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-17 04:24:52,951 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-17 04:24:52,952 : INFO : EPOCH - 9 : training on 1540 raw words (124 effective words) took 0.0s, 19746 effective words/s
2019-05-17 04:2

cluster file outputted!


({3: [0, 26],
  8: [1, 8, 10, 11, 12],
  6: [2],
  1: [3, 5, 14, 17],
  4: [4, 9, 18, 22],
  5: [6, 20, 24, 25],
  9: [7, 19, 23],
  7: [13],
  2: [15, 21, 27],
  0: [16]},
 {3: 67, 8: 84, 6: 513, 1: 96, 4: 46, 5: 36, 9: 103, 7: 14, 2: 14, 0: 27})